In [2]:
import numpy as np
import pandas as pd
from PIL import Image
from skimage import filters
import scipy.ndimage as sci_image
from skimage import measure as meas
import matplotlib.pyplot as plt
from scipy import signal as sci_signal
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
import os
import warnings
warnings.filterwarnings("ignore")

In [45]:
def detect_letters():
    
    character_dict = {}
    
    start_letter = ord("A")
    end_letter = ord("Z") + 1
    
    start_number = ord("0")
    end_number = ord("9") + 1
    
    total_character_codes = list(range(start_letter,end_letter)) + list(range(start_number,end_number))
    
    for i in total_character_codes:
        file_name = "data/" + chr(i) + ".bmp"
        image_read = cv2.imread(file_name)
        character_dict[chr(i)] = cv2.cvtColor(src=image_read, code=cv2.COLOR_BGR2GRAY)
    
    character_list = np.array(list(character_dict.values()))
    return character_list
    

In [46]:
print(detect_letters().shape)

(36, 42, 24)


In [ ]:
A=imread('alpha/A.bmp');B=imread('alpha/B.bmp');C=imread('alpha/C.bmp');
D=imread('alpha/D.bmp');E=imread('alpha/E.bmp');F=imread('alpha/F.bmp');
G=imread('alpha/G.bmp');H=imread('alpha/H.bmp');I=imread('alpha/I.bmp');
J=imread('alpha/J.bmp');K=imread('alpha/K.bmp');L=imread('alpha/L.bmp');
M=imread('alpha/M.bmp');N=imread('alpha/N.bmp');O=imread('alpha/O.bmp');
P=imread('alpha/P.bmp');Q=imread('alpha/Q.bmp');R=imread('alpha/R.bmp');
S=imread('alpha/S.bmp');T=imread('alpha/T.bmp');U=imread('alpha/U.bmp');
V=imread('alpha/V.bmp');W=imread('alpha/W.bmp');X=imread('alpha/X.bmp');
Y=imread('alpha/Y.bmp');Z=imread('alpha/Z.bmp');

%Natural Numbers
one=imread('alpha/1.bmp');two=imread('alpha/2.bmp');
three=imread('alpha/3.bmp');four=imread('alpha/4.bmp');
five=imread('alpha/5.bmp'); six=imread('alpha/6.bmp');
seven=imread('alpha/7.bmp');eight=imread('alpha/8.bmp');
nine=imread('alpha/9.bmp'); zero=imread('alpha/0.bmp');

%Creating Array for Alphabets
letter=[A B C D E F G H I J K L M N O P Q R S T U V W X Y Z];
%Creating Array for Numbers
number=[one two three four five six seven eight nine zero];

NewTemplates=[letter number];
save ('NewTemplates','NewTemplates')
clear all